In [0]:
import numpy as np
from scipy.stats import ttest_rel, ttest_ind
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import collections
warnings.filterwarnings("ignore")

# Data Creation
> wml = returns of WinnerMinusLoser portfolio <br>
> iml = returns of IlliquidMinusLiquid portfolio <br>
> market_value = market return <br>

In [0]:
wml_data = pd.read_csv('wml.csv', delimiter=',', header=0)
iml_data = pd.read_csv('iml.csv', delimiter=',', header=0)
target = pd.read_csv('returns.csv', delimiter=',', header=0)
market_value = pd.read_csv('market_value.csv', delimiter=',', header=0)

indeks = wml_data['Time'].tolist().index("01-Jan-2009 00:00:00")
indeks = iml_data['Time'].tolist().index("01-Jan-2009 00:00:00")

target = target[213:-1]

data = pd.DataFrame({'iml': iml_data['iml'][213:].dropna()})
new_col = pd.DataFrame({'wml': wml_data['wml'][213:].dropna()})
data = data.join(new_col)
data = data.reset_index(drop=True)
target = target.reset_index(drop=True)

market_volume = pd.DataFrame({'market_volume': market_value['Volume'][1:]})
market_volume = market_volume.reset_index(drop=True)

# XGBRegressor model
> X = feature set with names {market_volume(t-k)} where k from [1,6] month<br>
> Target = monthly stock returns<br>
> each stock has its own model<br>
> RMSE - measures the average error performed by the model in predicting the outcome for an observation

In [0]:
stocks = target.columns.values
volumes = []
for ind in range(0, 120):
  new_data = [0,1,2,3,4,5]
  for i in range(0, 6):
    new_data[i] = market_volume['market_volume'][ind+i]
  volumes.append(new_data)
#new_volumes = pd.DataFrame({'volume':volumes})
new_volumes = pd.DataFrame(columns=['volume_t-6', 'volume_t-5', 'volume_t-4', 'volume_t-3', 'volume_t-2', 'volume_t-1'], data=volumes)
#print(new_volumes)

X = new_volumes[['volume_t-6', 'volume_t-5', 'volume_t-4', 'volume_t-3', 'volume_t-2', 'volume_t-1']].values
data_matrixes = []
for Y in stocks[1:]:
    
    data_dmatrix = xgb.DMatrix(data=X,label=target[Y].values[1:])
    data_dmatrix.feature_names = ['volume_t-6', 'volume_t-5', 'volume_t-4', 'volume_t-3', 'volume_t-2', 'volume_t-1']
    data_matrixes.append(data_dmatrix)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, target[Y].values[1:], test_size=0.1, random_state=0, shuffle=False)
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
    xg_reg.fit(X_train,Y_train)

    preds = xg_reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
    print("---- %s ----- RMSE: %f" % (Y, rmse))


---- AA ----- RMSE: 0.302410
---- AB ----- RMSE: 0.108631
---- ABM ----- RMSE: 0.477598
---- ABT ----- RMSE: 0.097371
---- ABX ----- RMSE: 0.393036
---- ADM ----- RMSE: 0.264233
---- ADX ----- RMSE: 0.090802
---- AEM ----- RMSE: 0.305888
---- AEP ----- RMSE: 0.229757
---- AET ----- RMSE: 0.182621
---- AFG ----- RMSE: 0.163634
---- AFL ----- RMSE: 0.112405
---- AGN ----- RMSE: 0.555420
---- AIG ----- RMSE: 0.387311
---- AIN ----- RMSE: 0.117306
---- AIR ----- RMSE: 0.149734
---- AIT ----- RMSE: 0.125210
---- AJG ----- RMSE: 0.095876
---- AJRD ----- RMSE: 0.270159
---- ALE ----- RMSE: 0.216102
---- ALK ----- RMSE: 0.630956
---- ALX ----- RMSE: 0.319626
---- AME ----- RMSE: 0.135824
---- AON ----- RMSE: 0.232901
---- AOS ----- RMSE: 0.310648
---- AP ----- RMSE: 0.526365
---- APA ----- RMSE: 0.276371
---- APB ----- RMSE: 0.195770
---- APC ----- RMSE: 0.347268
---- APD ----- RMSE: 0.143445
---- ARL ----- RMSE: 0.548377
---- ARNC ----- RMSE: 0.358931
---- ARW ----- RMSE: 0.263600
---- ASA --

In [0]:
# function for model measuring
def compare_rmse(train_value, test_value):
  if (abs(test_value - train_value)) < 0.2:
    return "Model is good"
  else: return "Model is OVERFITTED or UNDERFITTED"

# CROSS VALIDATION OF XGBRegressor model
- parameters set randomly
- measuring with RMSE - if train and test values are similar than model is good
- kfold = 5

In [0]:
# cross-validation
for indeks, Y in enumerate(stocks[1:]):
  params = {"objective":"reg:squarederror",'colsample_bytree': 0.8,'learning_rate': 0.1,
                  'max_depth': 5, 'alpha': 10}

  cv_results = xgb.cv(dtrain=data_matrixes[indeks], params=params, nfold=5,
                      num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
  print("------ " + Y + " ------")
  print("Compare RMSE_TRAIN and RMSE_TEST")
  print(cv_results["test-rmse-mean"].iloc[-1])
  print(compare_rmse((cv_results["train-rmse-mean"].iloc[-1]), (cv_results["test-rmse-mean"].iloc[-1])))

------ AA ------
Compare RMSE_TRAIN and RMSE_TEST
0.4205924
Model is good
------ AB ------
Compare RMSE_TRAIN and RMSE_TEST
0.38347500000000007
Model is good
------ ABM ------
Compare RMSE_TRAIN and RMSE_TEST
0.23681659999999996
Model is good
------ ABT ------
Compare RMSE_TRAIN and RMSE_TEST
0.1850036
Model is good
------ ABX ------
Compare RMSE_TRAIN and RMSE_TEST
0.49812779999999995
Model is good
------ ADM ------
Compare RMSE_TRAIN and RMSE_TEST
0.2266416
Model is good
------ ADX ------
Compare RMSE_TRAIN and RMSE_TEST
0.18820180000000003
Model is good
------ AEM ------
Compare RMSE_TRAIN and RMSE_TEST
0.3976722
Model is good
------ AEP ------
Compare RMSE_TRAIN and RMSE_TEST
0.17747960000000002
Model is good
------ AET ------
Compare RMSE_TRAIN and RMSE_TEST
0.257573
Model is good
------ AFG ------
Compare RMSE_TRAIN and RMSE_TEST
0.2110768
Model is good
------ AFL ------
Compare RMSE_TRAIN and RMSE_TEST
0.3318746
Model is good
------ AGN ------
Compare RMSE_TRAIN and RMSE_TEST
0.

In [0]:
# find the most important feature in model
#Redundantno s obzirom na korištenje samo jednog fičra, ubacio jer sam znatiželjan hehe :)
importance_scores = []
for indeks, Y in enumerate(stocks[1:]):
  xg_reg = xgb.train(params=params, dtrain=data_matrixes[indeks], num_boost_round=10)
  #xgb.plot_tree(xg_reg,num_trees=0)
  #plt.rcParams['figure.figsize'] = [50, 10]
  #plt.show()
  importance_scores.append(xg_reg.get_score())

t6_importance = 0
t5_importance = 0
t4_importance = 0
t3_importance = 0
t2_importance = 0
t1_importance = 0
for score in importance_scores:
  for key in score:
      if(key=='volume_t-6'):
        t6_importance+=score[key]
      if(key=='volume_t-5'):
        t5_importance+=score[key]
      if(key=='volume_t-4'):
        t4_importance+=score[key]
      if(key=='volume_t-3'):
        t3_importance+=score[key]
      if(key=='volume_t-2'):
        t2_importance+=score[key]
      if(key=='volume_t-1'):
        t1_importance+=score[key]

t6_importance = t6_importance/len(stocks[1:])
t5_importance = t5_importance/len(stocks[1:])
t4_importance = t4_importance/len(stocks[1:])
t3_importance = t3_importance/len(stocks[1:])
t2_importance = t2_importance/len(stocks[1:])
t1_importance = t1_importance/len(stocks[1:])
print("Importance of volume t-6: " + str(t6_importance))
print("Importance of volume t-5: " + str(t5_importance))
print("Importance of volume t-4: " + str(t4_importance))
print("Importance of volume t-3: " + str(t3_importance))
print("Importance of volume t-2: " + str(t2_importance))
print("Importance of volume t-1: " + str(t1_importance))

Importance of volume t-6: 2.3964912280701753
Importance of volume t-5: 2.473684210526316
Importance of volume t-4: 2.219298245614035
Importance of volume t-3: 1.4543859649122808
Importance of volume t-2: 1.4807017543859649
Importance of volume t-1: 2.2719298245614037
